Create a "debiased matrix" (on the axis of gender) in which the shapes of male and female distributions are the same (excluding appropriately gendered words)

In [1]:
from gensim.models import Word2Vec
import logging
import math
import numpy as np
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import gensim.downloader as api
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
import ot
# necessary for 3d plot even if not used
from mpl_toolkits.mplot3d import Axes3D  # noqa
from matplotlib.collections import PolyCollection
import json

In [ ]:
wiki_model = Word2Vec.load("english-wikipedia-articles-20170820-models/enwiki_2017_08_20_fasttext.model")


In [3]:
cvs = np.load('english-wikipedia-articles-20170820-models/enwiki_2017_08_20_fasttext.model.trainables.syn1neg.npy')


In [4]:
wvs = np.load('english-wikipedia-articles-20170820-models/enwiki_2017_08_20_fasttext.model.wv.vectors.npy')

In [5]:
def build_prob_matrix(wvs,cvs,s):
    prob_matrix = np.empty([s,s])
    for i,v_wo in enumerate(wvs[:s]):
      for j,v_wi in enumerate(cvs[:s]):
        prob_matrix[i,j] = math.exp((np.dot(v_wo,v_wi)))
    return prob_matrix

We'll want to compare the debiased word embedding on some of the baseline WEATs, including things like insects vs flowers as well as the gender bias tests. We don't want to create a matrix the size of the corpus right now, so we can check the indicies of these words to get a rough idea of how big it needs to be.

In [6]:
with open('baseline_tests.json') as test_file:
    test_json = json.load(test_file)

Investigate the large indices of words in the test file.

In [7]:
max_index = 0
for test, word_lists in test_json['tests'].items():
    for name, word_list in word_lists.items():
        if name in ['X','Y','A','B']:
            for w in word_list:
                if w in wiki_model.wv.vocab:
                    i = wiki_model.wv.vocab[w].index
                    if i > max_index:
                        print('max word so far: {}: {}'.format(w,i))
                        max_index = i
                else:
                    print('{} not in vocab'.format(w))

max word so far: poetry: 1764
Shakespeare not in vocab
max word so far: symphony: 2800
max word so far: math: 5491
max word so far: algebra: 7058
max word so far: calculus: 13143
max word so far: computation: 14976
max word so far: hers: 21000
Shakespeare not in vocab
Einstein not in vocab
NASA not in vocab
Amy not in vocab
Joan not in vocab
Lisa not in vocab
Sarah not in vocab
Diana not in vocab
Kate not in vocab
Ann not in vocab
Donna not in vocab
John not in vocab
Paul not in vocab
Mike not in vocab
Kevin not in vocab
Steve not in vocab
Greg not in vocab
Jeff not in vocab
Bill not in vocab
max word so far: harpoon: 37904
max word so far: slingshot: 45438
max word so far: bagpipe: 45747
max word so far: stink: 50596
max word so far: pollute: 92998
bedbug not in vocab
blackfly not in vocab
horsefly not in vocab
gladiola not in vocab
max word so far: zinnia: 141733


It looks like we can use max index of around 22000

In [ ]:
# size = 22000
# orig_prob_matrix = build_prob_matrix(wvs,cvs,size)


In [ ]:
# normed_matrix = orig_prob_matrix/orig_prob_matrix.sum(axis=0)

In [9]:
normed_matrix = np.load('normed_matrix.npy')

In [46]:
# np.save('normed_matrix',normed_matrix)

We can investigate what the 'she' and 'he' distributions look like for example.

In [47]:
she = wiki_model.wv.vocab["she"].index
he = wiki_model.wv.vocab["he"].index

In [48]:
she_dist = normed_matrix[:,she]

In [49]:
he_dist = normed_matrix[:,he]

In [10]:
appropriately_gendered_words = ["he", "his", "her", "she", "him", "man", "women", "men", "woman", "spokesman", "wife", "himself", "son", "mother", "father", "chairman",
"daughter", "husband", "guy", "girls", "girl", "boy", "boys", "brother", "spokeswoman", "female", "sister", "male", "herself", "brothers", "dad",
"actress", "mom", "sons", "girlfriend", "daughters", "lady", "boyfriend", "sisters", "mothers", "king", "businessman", "grandmother",
"grandfather", "deer", "ladies", "uncle", "males", "congressman", "grandson", "bull", "queen", "businessmen", "wives", "widow",
"nephew", "bride", "females", "aunt", "prostate cancer", "lesbian", "chairwoman", "fathers", "moms", "maiden", "granddaughter",
"younger brother", "lads", "lion", "gentleman", "fraternity", "bachelor", "niece", "bulls", "husbands", "prince", "colt", "salesman", "hers",
"dude", "beard", "filly", "princess", "lesbians", "councilman", "actresses", "gentlemen", "stepfather", "monks", "ex girlfriend", "lad",
"sperm", "testosterone", "nephews", "maid", "daddy", "mare", "fiance", "fiancee", "kings", "dads", "waitress", "maternal", "heroine",
"nieces", "girlfriends", "sir", "stud", "mistress", "lions", "estranged wife", "womb", "grandma", "maternity", "estrogen", "ex boyfriend",
"widows", "gelding", "diva", "teenage girls", "nuns", "czar", "ovarian cancer", "countrymen", "teenage girl", "penis", "bloke", "nun",
"brides", "housewife", "spokesmen", "suitors", "menopause", "monastery", "motherhood", "brethren", "stepmother", "prostate",
"hostess", "twin brother", "schoolboy", "brotherhood", "fillies", "stepson", "congresswoman", "uncles", "witch", "monk", "viagra",
"paternity", "suitor", "sorority", "macho", "businesswoman", "eldest son", "gal", "statesman", "schoolgirl", "fathered", "goddess",
"hubby", "stepdaughter", "blokes", "dudes", "strongman", "uterus", "grandsons", "studs", "mama", "godfather", "hens", "hen", "mommy",
"estranged husband", "elder brother", "boyhood", "baritone", "grandmothers", "grandpa", "boyfriends", "feminism", "countryman",
"stallion", "heiress", "queens", "witches", "aunts", "semen", "fella", "granddaughters", "chap", "widower", "salesmen", "convent",
"vagina", "beau", "beards", "handyman", "twin sister", "maids", "gals", "housewives", "horsemen", "obstetrics", "fatherhood",
"councilwoman", "princes", "matriarch", "colts", "ma", "fraternities", "pa", "fellas", "councilmen", "dowry", "barbershop", "fraternal",
"ballerina"]

In [11]:
def get_indices_in_matrix(words):
    def get_index(w):
        try: 
            return wiki_model.wv.vocab[w].index
        except:
            print("not in model: {}".format(w))
            return size+10
    indices = list(map(lambda word: get_index(word), words))
    indices = list(filter(lambda i: i < size, indices))
    return np.flip(np.sort(indices))

In [12]:
app_gen_indices = get_indices_in_matrix(appropriately_gendered_words)

not in model: prostate cancer
not in model: younger brother
not in model: ex girlfriend
not in model: estranged wife
not in model: ex boyfriend
not in model: teenage girls
not in model: ovarian cancer
not in model: teenage girl
not in model: twin brother
not in model: eldest son
not in model: hubby
not in model: estranged husband
not in model: elder brother
not in model: twin sister


In [13]:
app_gen_indices

array([21353, 21000, 20573, 20493, 20237, 20001, 18705, 17899, 17859,
       17774, 17720, 17561, 17256, 16744, 16600, 16556, 16167, 14873,
       14675, 14537, 14523, 13817, 13688, 13607, 13540, 13451, 13343,
       13190, 11964, 11867, 11788, 11592, 11222, 10470, 10387, 10383,
       10185, 10115,  9883,  9813,  9554,  9280,  9236,  9226,  9179,
        9155,  9109,  9087,  8986,  7633,  7617,  7553,  7510,  7498,
        7455,  7259,  7212,  7131,  7034,  6963,  6904,  6642,  6325,
        6308,  6237,  6084,  5674,  5584,  5428,  5370,  5092,  5019,
        4577,  4144,  4068,  4044,  4021,  3790,  3524,  3349,  3126,
        3122,  3005,  2944,  2710,  2704,  2626,  2471,  2385,  2127,
        2091,  1748,  1552,  1467,  1443,  1250,  1243,  1218,  1204,
        1158,  1102,  1077,  1051,  1033,  1020,  1002,   992,   847,
         822,   789,   682,   679,   629,   627,   591,   551,   526,
         435,   402,   386,   306,   293,   276,    94,    42,    40,
          17,    13]

In [14]:
inapp_gen_indices = np.array((filter(lambda x: x not in app_gen_indices, np.arange(size))))
inapp_gen_indices

array([    0,     1,     2, ..., 21997, 21998, 21999])

Portion of the distribution in the app_gen section of the 'he' vector

In [54]:
np.sum(he_dist[app_gen_indices])

0.00012946995137729498

Portion of the distribution in the app_gen section of the 'she' vector

In [55]:
np.sum(she_dist[app_gen_indices])

0.2711311975090502

In [15]:
gender_word_pairs = [('he','she'),('man','woman'),('his','her'),('himself','herself'),('men','women'),('husband','wife'),('girl','boy'),('men','women'),('brother','sister'),('mother','father'),('aunt','uncle'),('grandfather','grandmother'),('son','daughter')]


In [19]:
def debias_vector_pairs(gender_pairs,orig_matrix,model):
    new_mat = orig_matrix.copy()
    for p1,p2 in gender_pairs:
        print('debiasing distributions {} and {}'.format(p1,p2))
        p1_index,p2_index = model.wv.vocab[p1].index, model.wv.vocab[p2].index
        p1_dist = new_mat[inapp_gen_indices,p1_index]
        p2_dist = new_mat[inapp_gen_indices,p2_index]
        normed_p1_dist,normed_p2_dist = p1_dist/np.sum(p1_dist), p2_dist/np.sum(p2_dist)
        A = np.vstack((normed_p1_dist, normed_p2_dist)).T
        n_distributions = A.shape[1]
        M = ot.utils.dist0(len(p1_dist))
        M /= M.max()
        alpha = 0.2  # 0<=alpha<=1
        weights = np.array([1 - alpha, alpha])

        # wasserstein
        reg = 1e2
        bary_wass = ot.bregman.barycenter(A, M, reg, weights)

        new_mat[inapp_gen_indices,p1_index] = bary_wass * np.sum(p1_dist)
        new_mat[inapp_gen_indices,p2_index] = bary_wass * np.sum(p2_dist)
    return new_mat



In [95]:
normed_matrix = np.load('normed_matrix.npy')

Caution -- the cell below takes ~1 hour to run.

In [21]:
new_mat = debias_vector_pairs(gender_word_pairs,normed_matrix,wiki_model)

In [22]:
np.save('debiased_matrix',new_mat)